# Air Temperature Mapping Demo

In [ ]:
import numpy as np
import ipywidgets as widgets
from ipyleaflet import Map, ImageOverlay
import rasterio as rio


def model_airtemp(solar_irradiance, ndvi, ndbi, ndwi, c=25, ndvi_beta=-3, ndbi_beta=4, ndwi_beta=-2):
    airtemp =  ndvi_beta*ndvi + ndbi_beta*ndbi + ndwi_beta*ndwi + np.random.normal(-1, 1) + c + solar_irradiance
    return airtemp

ndvi_array = rio.open('data/ndvi.tif').read(1)
ndwi_array = rio.open('data/ndwi.tif').read(1)
ndbi_array = rio.open('data/ndbi.tif').read(1)

image_extent = rio.open('data/ndvi.tif').bounds
SW = (bounds.left, bounds.bottom)
NE = (bounds.right, bounds.top)
bounds_tuple = (SW, NE)


def array_to_jpeg(array, jpeg_path='temp.jpeg'):
    array = np.moveaxis(airtemp_array, 0, -1)

    array_max = np.max(array)
    array_min = np.min(array)
    array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
    array = array.as_type(np.uint8)

    image = Image.fromarray(array)

    jpeg_path_temp = SAVE_FILE_FUNCTION(PTH)
    return jpeg_path


solar_irradiance_slider = widgets.FloatSlider(value=14, min=10, max=20, step=0.1, description='Solar Irradiance')
ndvi_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.01, description='NDVI Coefficient')
ndbi_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.01, description='NBVI Coefficient')
ndwi_slider = widgets.FloatSlider(value=0, min=-5, max=5, step=0.01, description='NBWI Coefficient')

def updateMap(change):

    new_airtemp = model_airtemp(solar_irradiance_slider.value, ndvi_array, ndbi_array, ndwi_array, 13, ndvi_slider.value, 
                  ndbi_slider.value, ndwi_slider.value)
    
    path = array_to_jpeg(new_airtemp)
    imageLayer = ImageOverlay(path, bounds_tuple)
    m.reset_layers()
    m.add_layer(imageLayer)
    

solar_irradiance_slider.observe(updateMap, 'value')
ndvi_slider.observe(updateMap, 'value')
ndbi_slider.observe(updateMap, 'value')
ndwi_slider.observe(updateMap, 'value')


irradiance_slider_container = widgets.Box([solar_irradiance_slider])
parameters_container = widgets.VBox([ndvi_slider, ndbi_slider, ndwi_slider])

controls_container = widgets.VBox([irradiance_slider_container, parameters_container])
mapDisplay = widgets.output()

mapLayout = widgets.HBox([mapDisplay, controls_container])

with mapLayout:
    display(m)

display(mapLayout)
